# Semantic role labeling with BERT

In this notebook, you'll perform semantic role labeling with BERT, using the Universal Propbank dataset.

## Before you begin

### Install libraries

Uncomment and run the following cells to install the required pip packages, for example when running the notebook in [colab](https://colab.research.google.com).

In [ ]:
# !pip install datasets
# !pip install seqeval
# !pip install accelerate==0.21.0
# !pip install transformers[torch]
# !pip install accelerate -U

### Import libraries

In [ ]:
import time
import pandas as pd
import transformers
import numpy as np
from transformers import AutoTokenizer,AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_metric
from datasets import Dataset
from utils import read_data_as_sentence,map_labels_in_dataframe,tokenize_and_align_labels,get_label_mapping,get_labels_from_map,load_srl_model,load_dataset,compute_metrics,write_predictions_to_csv,compute_evaluation_metrics_from_csv

## Step 1: Preprocess data

Before you can train the model, you need to extract sentences from the training, development and test datasets, and preprocess the sentences.

To preprocess the datase and save the resulting DataFrame to a file, call the `read_data_as_sentence()` function, including:

| Parameter name     | Required | Parameter description |
|--------------------|:--------------:|-------------|
| *positional 1*                   | ✅️ | The filepath for the CoNNLU dataset. |
| *positional 2*                 | ✅ | The filepath to write the preprocessed DataFrame to. |

In [ ]:
train_data = read_data_as_sentence('data/en_ewt-up-train.conllu', 'data/en_ewt-up-train.preprocessed.csv')
dev_data = read_data_as_sentence('data/en_ewt-up-dev.conllu', 'data/en_ewt-up-dev.preprocessed.csv')
test_data = read_data_as_sentence('data/en_ewt-up-test.conllu', 'data/en_ewt-up-test.preprocessed.csv')

The `read_data_as_sentence()` function returns DataFrames, where each row represents a sentence from the dataset passed to the function. Each sentence has been expanded based on its predicates, resulting in multiple copies of the same sentence, each focused on a different predicate.

The DataFrame has two columns:

- `input_form`: a list of strings, where each string represents a words in the sentence, followed by two special tokens:
    1. A special token (`[SEP]`), which denotes the separation between the words of the sentence and the predicate form. 
    2. The predicate form, which corresponds to the `argument` values for the same row in the DataFrame.
- `argument`: a list of strings, representing the arguments associated with each word in the sentence. The length of each list is equal to the number of words in the sentence, plus two additional elements, for the special token and predicate form. The arguments match the predicate appended to the `input_form` for the same row in the DataFrame.

### Explore the DataFrame

To explore the DataFrame, print the head of the preprocessed DataFrame:

In [37]:
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4799 entries, 0 to 4798
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   input_form  4799 non-null   object
 1   argument    4799 non-null   object
dtypes: object(2)
memory usage: 75.1+ KB
None


The **Non-Null** count for both columns should match, indicating there are as many lists of `input_form` values as there are lists of `argument` values.

Next, print the words and their argument labels for the first 20 sentences of the test dataset:

In [48]:
for form, argument in zip(test_data.input_form[:20], test_data.argument[:20]):
    for f, a in zip(form, argument):
        if f == '[SEP]':
            print('-' * 40)
        print(f"form: {f:<15} argument: {a}")
    print('\n' + '=' * 40 + '\n')

form: What            argument: _
form: if              argument: _
form: Google          argument: ARG1
form: Morphed         argument: _
form: Into            argument: _
form: GoogleOS        argument: ARG2
form: ?               argument: _
----------------------------------------
form: [SEP]           argument: None
form: Morphed         argument: None


form: What            argument: _
form: if              argument: _
form: Google          argument: ARG0
form: expanded        argument: _
form: on              argument: _
form: its             argument: _
form: search          argument: _
form: -               argument: _
form: engine          argument: _
form: (               argument: _
form: and             argument: _
form: now             argument: _
form: e-mail          argument: _
form: )               argument: _
form: wares           argument: ARG1
form: into            argument: _
form: a               argument: _
form: full            argument: _
form: -              

## Step 2: Import the BERT model and tokenizer

Use HuggingFace's [`AutoTokenizer`](https://huggingface.co/docs/transformers/v4.38.2/en/model_doc/auto#transformers.AutoTokenizer) to construct a DistilBERT tokenizer, which is based on the WordPiece algorithm. 

In [ ]:
# Set the model ID to use
model_id = "distilbert-base-uncased"

# Initialize the tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Check the assertion that the tokenizer is an instance of transformers.PreTrainedTokenizerFast 
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Checking the sentence representation

In [53]:
example = test_data['input_form'][1]
print(example)

['What', 'if', 'Google', 'expanded', 'on', 'its', 'search', '-', 'engine', '(', 'and', 'now', 'e-mail', ')', 'wares', 'into', 'a', 'full', '-', 'fledged', 'operating', 'system', '?', '[SEP]', 'expanded']


The sentence contains the [SEP] special token followed by the predicate. Therefore, the parameter `add_special_tokens` is set to True so that the index is converted to 102 accordingly and is not treated as another word. \
In addition, the sentence is already split into tokens, to the parameter `is_split_into_words` is also set to True

In [ ]:
tokenizer(example,add_special_tokens=True,is_split_into_words=True)

In [ ]:
tokenized_input = tokenizer(example,add_special_tokens=True,is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

## Tokenizing and preparing input for the model

Getting the mapping of all possible arguments across all datasets to a numerical value with the `get_label_mapping` function.\
None value stays as None to be mapped to the special token in the model.

In [ ]:
label_map = get_label_mapping(train_data, test_data, dev_data)

In [ ]:
print(label_map)

Converting the labels in the df to numerical values for the language model with `map_labels_in_dataframe` function. The label_map dictionary from the function above is needed to map the arguments to their value.\
Add a new column to the df matching the arguments to label numbers. 0 stands for '_' (no argument) and the rest of the arguments are alphabetically ordered. \
*None* label will be mapped to the *[SEP]* token.


In [ ]:
train_data = map_labels_in_dataframe(train_data,label_map)
dev_data = map_labels_in_dataframe(dev_data,label_map)
test_data = map_labels_in_dataframe(test_data,label_map)

Checking the head to confirm the labels were correctly converted:

In [ ]:
test_data.head()

Use `tokenize_and_align_labels` function to tokenize train, test, and dev dataframe. Padding is applied to make sure all input is the same length for the model.

In [ ]:
tokenized_test = tokenize_and_align_labels(tokenizer, test_data, label_all_tokens=True)
tokenized_train = tokenize_and_align_labels(tokenizer, train_data, label_all_tokens=True)
tokenized_dev = tokenize_and_align_labels(tokenizer, dev_data, label_all_tokens=True)

The input for the model has the corresponding special token [CLS] followed by the tokenized sentence, the special token [SEP], the predicate and the final [SEP] token. \
The numerical labels to be fed to the model correspond to the tokenized sentence.\
The input is padded so that every vector is of the same length, including the labels and the attention mask.

In [ ]:
print(tokenizer.convert_ids_to_tokens(tokenized_test["input_ids"][0]))
print(tokenized_test["attention_mask"][0])
print(tokenized_test["input_ids"][0])
print(tokenized_test["labels"][0])

Confirming all the tokens contain a label and the attention mask also matches the length of the input

In [ ]:
print(len(tokenized_test["input_ids"][0]),len(tokenized_test["labels"][0]),len(tokenized_test["attention_mask"][0]))

Converting the tokenized data to datasets format with the function `load_dataset`

In [ ]:
dataset_train = load_dataset(tokenized_train)
dataset_dev = load_dataset(tokenized_dev)
dataset_test = load_dataset(tokenized_test)

## Fine-tuning the model

Reducing the size of the dataset for a mini test

In [ ]:
small_train_dataset = dataset_train.shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset_dev.shuffle(seed=42).select(range(1000))
small_test_dataset = dataset_test.shuffle(seed=42).select(range(1000))

Getting the labels that will be predicted by the model with the `get_labels_from_map`function

In [ ]:
label_list = get_labels_from_map(label_map)

Loading the model for semantic role labelling task with function `load_srl_model` to get the model, its name and the arguments necessary for training. \
The model selected is **distilbert-base-uncased**

In [ ]:
model, model_name, args = load_srl_model(model_id, label_list)

In [ ]:
print(model_name)

Load the seqeval metric to compute the metrics from the predictions

In [ ]:
metric = load_metric("seqeval")

metric.compute(predictions=[label_list], references=[label_list])

Passing the arguments along with the datasets to the `trainer` function to fine-tune the model for semantic role labelling with `trainer.train()`

In [ ]:
trainer = Trainer(
        model,
        args,
        train_dataset=dataset_train,
        eval_dataset=dataset_dev,
        tokenizer=tokenizer,
        compute_metrics=lambda p: compute_metrics(*p, label_list, metric)
    )
trainer.train()

Evaluate a model fine-tuned for semantic role labelling with `trainer.evaluate()`

In [ ]:
trainer.evaluate()

After training is finished, the precision/recall/f1 for each category can be computed. \
The same function `compute_metrics` is applied on the result of the predict method.

In [ ]:
predictions, labels, _ = trainer.predict(dataset_test)
results = compute_metrics(predictions, labels, label_list, metric)
results

Writing the predictions together with the gold labels to a csv file with the function `write_predictions_to_csv` so that the metrics per class can be computed with the `compute_evaluation_metrics_from_csv` function.

In [ ]:
results_file = "predictions.csv"
write_predictions_to_csv(predictions, labels, label_list, results_file)
f1,classification_report = compute_evaluation_metrics_from_csv("predictions.csv")
print(classification_report)

Then, we save fine-tuned model.

In [ ]:
# Use these codes to save model:
tokenizer.save_pretrained("tokenizer.save_pretrained.distillbert-base-uncased-finetuned-srl")
trainer.save_model("trainer.save_model.distillbert-base-uncased-finetuned-srl")
model.save_pretrained("model.save_pretrained.distillbert-base-uncased-finetuned-srl")


Here, we copy saved model to google drive.

In [ ]:
!cp -r '/content/trainer.save_model.distillbert-base-uncased-finetuned-srl' '/content/drive/MyDrive/NLP_3_baseline_model/model'
!cp -r '/content/model.save_pretrained.distillbert-base-uncased-finetuned-srl' '/content/drive/MyDrive/NLP_3_baseline_model/model'
!cp -r '/content/tokenizer.save_pretrained.distillbert-base-uncased-finetuned-srl' '/content/drive/MyDrive/NLP_3_baseline_model/model'